In [54]:
import json
import nltk
import string
import numpy as np
import pickle
from keras.src.models import Sequential
from keras.src.layers import Dense, Dropout, Input

np.set_printoptions(suppress=True)

In [6]:
with open("data/intents.json") as f:
    intents = json.load(f)["intents"]

In [ ]:
unique_words = set()
unique_classes = set()
document = []
lemmatizer = nltk.WordNetLemmatizer()
ignore_words = string.punctuation
ignore_words

In [8]:
for intent in intents:
    for pattern in intent["patterns"]:
        word_list = nltk.word_tokenize(pattern)
        word_list = [lemmatizer.lemmatize(word.lower()) for word in word_list if word not in ignore_words]
        unique_words.update(word_list)
        unique_classes.add(intent["tag"])
        document.append((word_list, intent["tag"]))

In [9]:
unique_words = sorted(unique_words)
unique_classes = sorted(unique_classes)

In [55]:
with open("data/words.pkl", "wb") as f:
    pickle.dump(unique_words, f)

with open("data/classes.pkl", "wb") as f:
    pickle.dump(unique_classes, f)

In [10]:
trainX = []
trainY = []

In [11]:
for pair in document:
    features = [0] * len(unique_words)
    for word in pair[0]:
        features[unique_words.index(word)] = 1
    trainX.append(np.array(features))
    
    labels = [0] * len(unique_classes)
    labels[unique_classes.index(pair[1])] = 1
    trainY.append(np.array(labels))

In [ ]:
model = Sequential([
    Input(shape=(len(trainX[0]),)),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(len(trainY[0]), activation="softmax")
])
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5)